<img src="https://tu-dortmund.sciebo.de/s/UGnVk4sv4ryF9ZF/download" height="100" align="center" style="height:180px"> 

<img src="https://bmrb.io/images/BMRB_logo_icon.svg" height="100" align="left" style="height:60px">

# BRMB vis
This is a tool to visualise assignments from [BMRB](https://bmrb.io/). Simply run all cells below and follow the instructions!

In [ ]:
#@title Install dependencies {display-mode: "form"}
%%bash

pip install -q --no-warn-conflicts pynmrstar

In [ ]:
#@title Import libraries and fetching the data {display-mode: "form"}

import pynmrstar
import pandas as pd
import matplotlib.pyplot as plt
import sys
from io import StringIO

#@markdown Enter the BMRB ID.

#@markdown For example, `51228` for Alex's SH3 assignments

try:
    bmrb_id = 51166 #@param {type:"integer"} 
    nmrstar = pynmrstar.Entry.from_database(bmrb_id)

except ValueError:
    print("Not a number")
except OSError:
    print(f"Entry {bmrb_id} does not exist in the public database")
    
nmrstar = pynmrstar.Entry.from_database(bmrb_id)
loops = nmrstar.get_saveframes_by_category('assigned_chemical_shifts')[0]
shifts = loops['_Atom_chem_shift']

## print(shifts)

data = StringIO(shifts.get_data_as_csv().replace("_Atom_chem_shift.", ""))
df = pd.read_csv(data, index_col=0)

## print(df.columns)

#### To see raw data from NMR Star file
#### print(df.tail(10))

# Rearranging the data

df1 = df["Auth_seq_ID Auth_comp_ID Auth_atom_ID Val Val_err Ambiguity_code".split()].set_index("Auth_seq_ID Auth_comp_ID Auth_atom_ID".split())
df1.columns = "ChemShift CS_error Ambiguity".split()
df1.index.names = ("ResNum", "ResType", "AtomType")
#@markdown Rearranged data from the NMR Star table

#@markdown Click on the Magic Wand button to the right and then the Filter button to search through the table
df1

In [ ]:
#@title Check for specific residues {display-mode: "form"}
#@markdown Show chemical shifts of the nuclei of the specific residue:
residue_number = 50 #@param {type:"integer"}
try:
    print(df1.loc[residue_number, :, :])
except KeyError:
    print("This residue does not have assignments or is not in the chain")

In [ ]:
#@title Visualisation {display-mode: "form"}
#@markdown Select chemical shifts for X and Y axes:

dimX = "H" #@param ["H", "N", "C", "CA", "CB", "HA"] {type:"string"}
dimY = "N" #@param ["H", "N", "C", "CA", "CB", "HA"] {type:"string"}

x = df1["ChemShift"].loc[:, :, dimX]
y = df1["ChemShift"].loc[:, :, dimY]

df_2D = pd.merge(x, y, on="ResNum", suffixes = (f"_{dimX}", f"_{dimY}"))
fig = df_2D.plot(x=f"ChemShift_{dimX}", y=f"ChemShift_{dimY}", kind="scatter", 
                 backend="plotly", text=df_2D.index,
          )

fig.update_xaxes(
    autorange="reversed")
fig.update_yaxes(
    autorange="reversed")
fig.update_traces(textposition="top left")
fig.update_layout(
    autosize=False,
    width=800,
    height=800)